# Importing necessary libraries

In [30]:
import pandas as pd
import numpy as np
import plotly.express as px
import random

# Importing all the flight data

In [31]:
file1 = pd.read_csv('data/msn_02_fuel_leak_signals_preprocessed.csv', sep=';')
file2 = pd.read_csv('data/msn_10_fuel_leak_signals_preprocessed.csv', sep=';')
file3 = pd.read_csv('data/msn_11_fuel_leak_signals_preprocessed.csv', sep=';')
file4 = pd.read_csv('data/msn_12_fuel_leak_signals_preprocessed.csv', sep=';')
file5 = pd.read_csv('data/msn_14_fuel_leak_signals_preprocessed.csv', sep=';')
file6 = pd.read_csv('data/msn_29_fuel_leak_signals_preprocessed.csv', sep=';')
file7 = pd.read_csv('data/msn_37_fuel_leak_signals_preprocessed.csv', sep=';')
file8 = pd.read_csv('data/msn_53_fuel_leak_signals_preprocessed.csv', sep=';')


# Creating 1 dataframe with all the data

In [32]:
#combine all files into one file
combined = pd.concat([file2, file3, file4, file5, file6, file7, file8])

# Feature Engineering and Data Enrichment

In this section, we perform feature engineering to create new variables that will help us extract more insightful information from our existing dataset. These new features are derived from the original variables and will be used to enhance our analysis and predictive modeling efforts. The following steps outline the creation of these new features:

1. **Total Fuel Used Calculation:**
   - Create a new column `total_fuel_used` as the sum of fuel used across all engines (`FUEL_USED_1`, `FUEL_USED_2`, `FUEL_USED_3`, `FUEL_USED_4`).

2. **Fuel On Board in Tanks Calculation:**
   - Create a new column `Fuel_On_board_tanks` by summing the fuel quantities in various tanks (`VALUE_FUEL_QTY_CT`, `VALUE_FUEL_QTY_FT1`, `VALUE_FUEL_QTY_FT2`, `VALUE_FUEL_QTY_FT3`, `VALUE_FUEL_QTY_FT4`, `VALUE_FUEL_QTY_LXT`, `VALUE_FUEL_QTY_RXT`).

3. **Fuel Used vs. Fuel On Board:**
   - Create a new column `FOB - total_fuel_used` to compare the initial fuel on board (`VALUE_FOB`) with the total fuel used.

4. **Fuel Usage Per Minute Calculation:**
   - Calculate the difference in `VALUE_FOB` and `total_fuel_used` between consecutive rows to get `fuel_used_per_minute_FOB_diff` and `fuel_used_per_minute_total_fuel_used_diff`.

5. **Cumulative Fuel Used Calculation:**
   - Compute the cumulative sum of `total_fuel_used` and store it in `cummalitive_fuel_used`.

6. **Initial Fuel On Board:**
   - Record the initial `VALUE_FOB` for each flight in `initial_FOB`.
   - Calculate the initial fuel on board from the tanks for each flight and store it in `initial_FOB_calculated`.

7. **Predicted Fuel On Board:**
   - Create `fuel_used+FOB` by adding `total_fuel_used` to `VALUE_FOB`.
   - Calculate the difference between `initial_FOB` and `fuel_used+FOB` and store it in `diff_initial_FOB_fuel_used+FOB`.
   - Predict the fuel on board using the initial `VALUE_FOB` and `total_fuel_used`, storing the result in `predicted_FOB`.
   - Compute the difference between `initial_FOB` and `initial_FOB_calculated` and store it in `diff_initial_FOB`.

8. **Altitude and Fuel Differences:**
   - Calculate the difference in altitude between consecutive rows and store it in `alititude_diff`.
   - Calculate the difference in `VALUE_FOB` between consecutive rows and store it in `diff_FOB`.

9. **Real vs. Predicted Fuel On Board:**
   - Calculate the difference between the real `VALUE_FOB` and the `predicted_FOB` and store it in `Real_FOB-predicted_FOB`.

These features will provide a comprehensive understanding of fuel usage patterns, discrepancies in fuel measurements, and other crucial insights for further analysis.


In [38]:
# Create a new column called total_fuel_used which is the sum of FUEL_USED_1, FUEL_USED_2, FUEL_USED_3, and FUEL_USED_4
combined['total_fuel_used'] = combined['FUEL_USED_1'] + combined['FUEL_USED_2'] + combined['FUEL_USED_3'] + combined['FUEL_USED_4']

# Create a new column called Fuel_On_board_tanks which is the sum of various fuel quantity measurements from different tanks
combined['Fuel_On_board_tanks'] = (combined['VALUE_FUEL_QTY_CT'] + combined['VALUE_FUEL_QTY_FT1'] + combined['VALUE_FUEL_QTY_FT2'] +
                                   combined['VALUE_FUEL_QTY_FT3'] + combined['VALUE_FUEL_QTY_FT4'] + combined['VALUE_FUEL_QTY_LXT'] + 
                                   combined['VALUE_FUEL_QTY_RXT'])

# Create a new column to calculate the difference between VALUE_FOB and total_fuel_used
combined['FOB - total_fuel_used'] = combined['VALUE_FOB'] - combined['total_fuel_used']

# Calculate the difference in VALUE_FOB between consecutive rows and store it in a new column fuel_used_per_minute_FOB_diff
combined['fuel_used_per_minute_FOB_diff'] = combined['VALUE_FOB'].diff()

# Calculate the difference in total_fuel_used between consecutive rows and store it in a new column fuel_used_per_minute_total_fuel_used_diff
combined['fuel_used_per_minute_total_fuel_used_diff'] = combined['total_fuel_used'].diff()

# Calculate the cumulative sum of total_fuel_used and store it in a new column cummalitive_fuel_used
combined['cummalitive_fuel_used'] = combined['total_fuel_used'].cumsum()

# Record the initial VALUE_FOB for each flight and store it in a new column initial_FOB
combined['initial_FOB'] = combined.groupby('Flight')['VALUE_FOB'].transform('first')

# Calculate the initial fuel on board from the tanks for each flight and store it in a new column initial_FOB_calculated
combined['initial_FOB_calculated'] = combined.groupby('Flight')['Fuel_On_board_tanks'].transform('first')

# Create a new column fuel_used+FOB by adding total_fuel_used to VALUE_FOB
combined['fuel_used+FOB'] = combined['total_fuel_used'] + combined['VALUE_FOB']

# Calculate the difference between initial_FOB and fuel_used+FOB and store it in a new column diff_initial_FOB_fuel_used+FOB
combined['diff_initial_FOB_fuel_used+FOB'] = combined['initial_FOB'] - combined['fuel_used+FOB']

# Predict the fuel on board by subtracting total_fuel_used from initial_FOB and store it in a new column predicted_FOB
combined['predicted_FOB'] = combined['initial_FOB'] - combined['total_fuel_used']

# Calculate the difference between initial_FOB and initial_FOB_calculated and store it in a new column diff_initial_FOB
combined['diff_initial_FOB'] = combined['initial_FOB'] - combined['initial_FOB_calculated']

# Calculate the difference in altitude between consecutive rows and store it in a new column alititude_diff
combined['alititude_diff'] = combined['FW_GEO_ALTITUDE'].diff()

# Calculate the difference in VALUE_FOB between consecutive rows and store it in a new column diff_FOB
combined['diff_FOB'] = combined['VALUE_FOB'].diff()

# Calculate the difference between VALUE_FOB and predicted_FOB and store it in a new column Real_FOB-predicted_FOB
combined['Real_FOB-predicted_FOB'] = combined['VALUE_FOB'] - combined['predicted_FOB']

# Handling Missing Values

In our dataset, we encountered a significant number of null values. After careful consideration, we decided to drop these null values rather than impute them. The reason for this decision is that imputing the missing values could lead to information loss and potentially introduce biases or inaccuracies in our analysis. By removing the null values, we ensure that our dataset remains robust and reliable for subsequent analysis and modeling.

Further details on this decision and its implications are explained in our report.

In [34]:
combined.dropna(inplace=True)

# Filtering Data by Flight Phase

For our analysis, we decided to retain only the data where the flight phase is 8. The rationale behind this decision is that during flight phase 8, the fuel amount data is most stable. This is because the engine consumption is at its most stable and the altitude change is minimal. By focusing on this flight phase, we can ensure the accuracy and reliability of our fuel consumption analysis.

Further details on this decision and its implications are explained in our report.


In [35]:
combined = combined[combined['FLIGHT_PHASE_COUNT'] == 8]

# Checking Remaining Unique Flights

After filtering the dataset to retain only the data where the flight phase is 8, we want to verify the number of unique flights that remain. Originally, our dataset included 2229 flights, including test flights. By focusing on the stable flight phase 8, we ensure the quality of our data. Despite this filtering, we still have a substantial amount of data, with 479 unique flights remaining for our analysis.

Further details on this decision and its implications are explained in our report.

In [36]:
combined['Flight'].nunique()

479

# Simulating Fuel Leakage in Flights

In this section, we simulate fuel leakage in a subset of flights to understand how it impacts fuel metrics. The following steps outline the process:

1. **Copy the Dataset:**
   - Make a copy of the preprocessed dataset to ensure the original data remains unchanged.

2. **Check for Required Column:**
   - Ensure the dataset contains the 'VALUE_FOB' column, which is essential for the simulation. If not present, raise an error.

3. **Define Leakage Simulation Function:**
   - Implement a function `simulate_fake_leakage` to simulate a fuel leak starting mid-flight. The function calculates the fuel loss over time and adjusts the `VALUE_FOB` accordingly to create a new column `FAKE_LEAKAGE`.

4. **Randomly Select Flights:**
   - Select 100 random flights from the dataset to apply the fake leakage simulation. A random seed is set for reproducibility.

5. **Apply Leakage Simulation:**
   - For each selected flight, determine a random starting point mid-flight to begin the leak. Apply the leakage simulation and update the `FAKE_LEAKAGE` column. Additionally, append an 'L' to the flight name to indicate that a leak has been simulated.

6. **Print Information:**
   - Print the flight numbers where the fake leakage has been implemented for verification.

7. **Export Modified Dataset:**
   - Save the modified dataset, which includes the simulated leakage, to a CSV file named `combined_leaks.csv`.

8. **Plot Fuel Usage Trends:**
   - Select one of the flights with simulated leakage at random. Plot the trend of fuel usage (`VALUE_FOB` and `FAKE_LEAKAGE`) over time to visualize the impact of the simulated leak.

Further details on the methodology and its implications are explained in our report.


In [37]:
import pandas as pd
import numpy as np
import plotly.express as px

# Assuming combined is your preprocessed dataset
combined = combined.copy()

# Ensure 'VALUE_FOB' is present in the dataset
if 'VALUE_FOB' not in combined.columns:
    raise KeyError("The dataset must contain the 'VALUE_FOB' column.")

# Function to simulate a leak starting mid-flight
def simulate_fake_leakage(flight_data, leak_rate, start_point):
    flight_data = flight_data.copy()
    leak_duration = len(flight_data) - start_point
    leak_loss = np.zeros(len(flight_data))
    leak_loss[start_point:] = np.arange(leak_duration) * leak_rate
    flight_data['FAKE_LEAKAGE'] = flight_data['VALUE_FOB'] - leak_loss
    flight_data['LEAK'] = 0
    flight_data.iloc[start_point:, flight_data.columns.get_loc('LEAK')] = 1
    return flight_data

# Randomly select flights to add fake leakage
random_flights = combined['Flight'].unique()
np.random.seed(42)  # For reproducibility
selected_flights = np.random.choice(random_flights, size=100, replace=False)  # Select 100 random flights

# Initialize FAKE_LEAKAGE column with VALUE_FOB values
combined['FAKE_LEAKAGE'] = combined['VALUE_FOB']
combined['LEAK'] = 0  # Initialize LEAK column

# Apply the fake leakage simulation to selected flights
leak_rate = 0.5  # Define a leak rate

for flight in selected_flights:
    flight_data = combined[combined['Flight'] == flight]
    start_point = np.random.randint(len(flight_data) // 2, len(flight_data))  # Start leak mid-flight
    simulated_data = simulate_fake_leakage(flight_data, leak_rate, start_point)
    combined.loc[combined['Flight'] == flight, ['FAKE_LEAKAGE', 'LEAK']] = simulated_data[['FAKE_LEAKAGE', 'LEAK']]

# Print out where the fake leakages were implemented
for flight in selected_flights:
    print(f"Fake leakage implemented in flight: {flight} starting from mid-flight.")

import random

# Select a flight at random
random_leak_flight = str(random.choice(selected_flights)) + 'L'

# Get the data for the selected flight
flight_data = combined[combined['Flight'] == random_leak_flight]

# Plot the data
fig = px.line(flight_data, x='UTC_TIME', y=['VALUE_FOB', 'FAKE_LEAKAGE'],
              labels={'value': 'Fuel (kg)', 'UTC_TIME': 'Time'},
              title=f'Trend of Fuel Usage with Simulated Leakage for Flight {random_leak_flight}')
fig.update_layout(legend_title_text='Fuel Metrics')
fig.show()


/var/folders/6g/x5lm6rt15m9bd1llv55ld0_h0000gn/T/ipykernel_7530/1640251842.py:28: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['389.0L' '389.0L' '389.0L' '389.0L' '389.0L' '389.0L' '389.0L' '389.0L'
 '389.0L' '389.0L' '389.0L' '389.0L' '389.0L' '389.0L' '389.0L' '389.0L'
 '389.0L' '389.0L' '389.0L' '389.0L' '389.0L' '389.0L' '389.0L' '389.0L'
 '389.0L' '389.0L' '389.0L' '389.0L' '389.0L' '389.0L' '389.0L' '389.0L'
 '389.0L' '389.0L' '389.0L' '389.0L' '389.0L' '389.0L' '389.0L' '389.0L'
 '389.0L' '389.0L' '389.0L' '389.0L' '389.0L' '389.0L' '389.0L' '389.0L'
 '389.0L' '389.0L' '389.0L' '389.0L' '389.0L' '389.0L' '389.0L' '389.0L'
 '389.0L' '389.0L' '389.0L' '389.0L' '389.0L' '389.0L' '389.0L' '389.0L'
 '389.0L' '389.0L' '389.0L' '389.0L' '389.0L' '389.0L' '389.0L' '389.0L'
 '389.0L' '389.0L' '389.0L' '389.0L' '389.0L' '389.0L' '389.0L' '389.0L'
 '389.0L' '389.0L' '389.0L' '389.0L' '389.0L' '389.0L' '38

Fake leakage implemented in flight: 389.0 starting from mid-flight.
Fake leakage implemented in flight: 139.0 starting from mid-flight.
Fake leakage implemented in flight: 1995.0 starting from mid-flight.
Fake leakage implemented in flight: 1882.0 starting from mid-flight.
Fake leakage implemented in flight: 166.0 starting from mid-flight.
Fake leakage implemented in flight: 217.0 starting from mid-flight.
Fake leakage implemented in flight: 109.0 starting from mid-flight.
Fake leakage implemented in flight: 363.0 starting from mid-flight.
Fake leakage implemented in flight: 28.0 starting from mid-flight.
Fake leakage implemented in flight: 1798.0 starting from mid-flight.
Fake leakage implemented in flight: 2135.0 starting from mid-flight.
Fake leakage implemented in flight: 256.0 starting from mid-flight.
Fake leakage implemented in flight: 421.0 starting from mid-flight.
Fake leakage implemented in flight: 67.0 starting from mid-flight.
Fake leakage implemented in flight: 104.0 star

In [39]:
# Create a new column called total_fuel_used which is the sum of FUEL_USED_1, FUEL_USED_2, FUEL_USED_3, and FUEL_USED_4
combined['total_fuel_used'] = combined['FUEL_USED_1'] + combined['FUEL_USED_2'] + combined['FUEL_USED_3'] + combined['FUEL_USED_4']

# Create a new column called Fuel_On_board_tanks which is the sum of various fuel quantity measurements from different tanks
combined['Fuel_On_board_tanks'] = (combined['VALUE_FUEL_QTY_CT'] + combined['VALUE_FUEL_QTY_FT1'] + combined['VALUE_FUEL_QTY_FT2'] +
                                   combined['VALUE_FUEL_QTY_FT3'] + combined['VALUE_FUEL_QTY_FT4'] + combined['VALUE_FUEL_QTY_LXT'] + 
                                   combined['VALUE_FUEL_QTY_RXT'])

# Create a new column to calculate the difference between VALUE_FOB and total_fuel_used
combined['FOB - total_fuel_used'] = combined['VALUE_FOB'] - combined['total_fuel_used']

# Calculate the difference in VALUE_FOB between consecutive rows and store it in a new column fuel_used_per_minute_FOB_diff
combined['fuel_used_per_minute_FOB_diff'] = combined['VALUE_FOB'].diff()

# Calculate the difference in total_fuel_used between consecutive rows and store it in a new column fuel_used_per_minute_total_fuel_used_diff
combined['fuel_used_per_minute_total_fuel_used_diff'] = combined['total_fuel_used'].diff()

# Calculate the cumulative sum of total_fuel_used and store it in a new column cummalitive_fuel_used
combined['cummalitive_fuel_used'] = combined['total_fuel_used'].cumsum()

# Record the initial VALUE_FOB for each flight and store it in a new column initial_FOB
combined['initial_FOB'] = combined.groupby('Flight')['VALUE_FOB'].transform('first')

# Calculate the initial fuel on board from the tanks for each flight and store it in a new column initial_FOB_calculated
combined['initial_FOB_calculated'] = combined.groupby('Flight')['Fuel_On_board_tanks'].transform('first')

# Create a new column fuel_used+FOB by adding total_fuel_used to VALUE_FOB
combined['fuel_used+FOB'] = combined['total_fuel_used'] + combined['VALUE_FOB']

# Calculate the difference between initial_FOB and fuel_used+FOB and store it in a new column diff_initial_FOB_fuel_used+FOB
combined['diff_initial_FOB_fuel_used+FOB'] = combined['initial_FOB'] - combined['fuel_used+FOB']

# Predict the fuel on board by subtracting total_fuel_used from initial_FOB and store it in a new column predicted_FOB
combined['predicted_FOB'] = combined['initial_FOB'] - combined['total_fuel_used']

# Calculate the difference between initial_FOB and initial_FOB_calculated and store it in a new column diff_initial_FOB
combined['diff_initial_FOB'] = combined['initial_FOB'] - combined['initial_FOB_calculated']

# Calculate the difference in altitude between consecutive rows and store it in a new column alititude_diff
combined['alititude_diff'] = combined['FW_GEO_ALTITUDE'].diff()

# Calculate the difference in VALUE_FOB between consecutive rows and store it in a new column diff_FOB
combined['diff_FOB'] = combined['VALUE_FOB'].diff()

# Calculate the difference between VALUE_FOB and predicted_FOB and store it in a new column Real_FOB-predicted_FOB
combined['Real_FOB-predicted_FOB'] = combined['VALUE_FOB'] - combined['predicted_FOB']

In [40]:
# Export the modified dataset to a CSV file
combined.to_csv('combined_leaks.csv', index=False)